In [ ]:
%matplotlib inline
from IPython.core.pylabtools import figsize

from functools import partial
import itertools
import re

import matplotlib.gridspec as gridspec
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pylab as pl
import scipy as sp
from graphviz import Digraph
import pydot
from IPython.display import Image
import seaborn as sns

print(__doc__)
from patsy import dmatrices
import scipy as sc
import scipy.optimize as opt
import statsmodels.api as sm

import sklearn as sk


from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import TransformerMixin
from sklearn.pipeline import BaseEstimator
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.utils.metaestimators import if_delegate_has_method
from sklearn.utils import column_or_1d
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union


from sklearn.datasets import samples_generator
from sklearn import datasets

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer

from sklearn.feature_extraction import DictVectorizer

import sklearn.feature_extraction as fe

import sklearn.feature_selection as fs
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import VarianceThreshold

import sklearn.linear_model as lm
import sklearn.grid_search as gs
from sklearn.svm import SVC
from sklearn.decomposition import PCA


import sklearn.preprocessing as preprocessing
import sklearn.cross_validation as cv
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc

from sklearn_pandas import DataFrameMapper, cross_val_score

import matplotlib.pyplot as plt
import patsy
from statsmodels.sandbox import cox

from patsylearn import *

import joblib as jb


from datasu.auc import *
from datasu.dicts import *
from datasu.files import *
from datasu.pandas import *
from datasu.persist import *

import pandas as pd

## UTILS

## LOAD DATA

In [ ]:
main_folder = '~/dev/data/Kagle-ValuesShoppers/'
df_coupons = pd.read_csv(main_folder+'offers')[['offer','category','company','brand','offervalue','quantity']]
df_offers_ids = pd.read_csv(main_folder+'trainHistory').rename(columns={'id': 'customer_id'})
df_offers_ids_submission = pd.read_csv(main_folder+'testHistory').rename(columns={'id': 'customer_id'})
df_trans_all = pd.read_csv(main_folder+'transactions_reduced_category').rename(columns={'id': 'customer_id'})

In [ ]:
df_offers_all = pd.merge(df_offers_ids, df_coupons, on=['offer'])
df_offers_all = df_offers_all[['customer_id','chain','offer','market','category','company','brand','offerdate','offervalue','quantity','repeattrips','repeater']]

df_offers_all_submission = pd.merge(df_offers_ids_submission, df_coupons, on=['offer'])
df_offers_all_submission = df_offers_all_submission[['customer_id','chain','offer','market','category','company','brand','offerdate','offervalue','quantity']]

In [ ]:
df_coupons.head()

In [ ]:
df_coupons[df_coupons.offer==1213242]

In [ ]:
df_offers_all.head()

In [ ]:
df_trans_all.head()

In [ ]:
offer_counts = df_offers_all.groupby(by='offer').count().customer_id.order(ascending=False)
sns.barplot(y=offer_counts.index.values, x=offer_counts.values, orient='h')

## FILTER DATA

In [ ]:
df_trans = df_trans_all#[0:3000000]

In [ ]:
df_offers_filtered = df_offers_all#[df_offers_all.offer==1197502]

In [ ]:
len(df_trans.customer_id.unique()), len(df_trans.customer_id),  len(df_offers_filtered.customer_id.unique()),len(df_offers_filtered.customer_id), 

## FEATURE EXTRACTION

### AGGREGATE TRANSCATIONS

In [ ]:
summ_grouping = {'total':np.sum, 'average':np.average }
count_grouping = {'count':np.count_nonzero }

In [ ]:
count_agg = partial(build_df_agg, agg_columns=['customer_id'], agg_funcs=count_grouping)
total_avg_agg = partial(build_df_agg, agg_columns=['productsize','purchasequantity','purchaseamount'], agg_funcs=summ_grouping)

In [ ]:
grpby_columns = ['customer_id','brand']

df_trans_grp_customer_brand = df_trans.groupby(by=grpby_columns, axis=0) \
                               .agg(merge_dicts(count_agg(grpby_columns), \
                                                total_avg_agg(grpby_columns)))    
    
df_trans_grp_customer_brand.columns = flatten_columns(df_trans_grp_customer_brand,prefix='agg_')
df_trans_grp_customer_brand = df_trans_grp_customer_brand.reset_index()#.set_index('customer_id')
df_trans_grp_customer_brand.head()


In [ ]:
print len(df_trans_grp_customer_brand.brand.unique())
print len(df_trans_grp_customer_brand)

In [ ]:
grpby_columns = ['customer_id','category']
grp_columns = ['purchaseamount']

df_trans_grp_customer_category = df_trans.groupby(by=grpby_columns, axis=0) \
                                         .agg(merge_dicts(count_agg(grpby_columns), \
                                              total_avg_agg(grpby_columns))) 
    
df_trans_grp_customer_category.columns = flatten_columns(df_trans_grp_customer_category, prefix='agg_')
df_trans_grp_customer_category = df_trans_grp_customer_category.reset_index()#.set_index('customer_id')
df_trans_grp_customer_category.head()



In [ ]:
print len(df_trans_grp_customer_category)

In [ ]:
grpby_columns = ['customer_id','dept']
grp_columns = ['productsize','purchaseamount']

df_trans_grp_customer_dept = df_trans.groupby(by=grpby_columns, axis=0) \
                                     .agg(merge_dicts(count_agg(grpby_columns), \
                                          total_avg_agg(grpby_columns))) 
    
df_trans_grp_customer_dept.columns = flatten_columns(df_trans_grp_customer_dept, prefix='agg_')
df_trans_grp_customer_dept = df_trans_grp_customer_dept.reset_index()#.set_index('customer_id')
df_trans_grp_customer_dept.head()

### MERGE FEATURES TO ONE TABLE

In [ ]:
df_offers__trans_aggs = merge_data(df_offers_all, how='left',
                                  rights=[(df_trans_grp_customer_brand,['customer_id','brand']),
                                          (df_trans_grp_customer_category,['customer_id', 'category']),
#                                           (df_trans_grp_customer_dept,['customer_id'])
                                         ])

df_offers__trans_aggs.fillna(0, inplace=True)
df_offers__trans_aggs.head()

### PERSIST DATA TO DISK

In [ ]:
persist_variables(['df_trans_grp_customer_brand',
                   'df_trans_grp_customer_category',
                   'df_trans_grp_customer_dept',
                   'df_offers__trans_aggs']
                  ,path=main_folder+'working_data')